In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [39]:
URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(URL, index_col=False)
passengers = data.Passengers
months = data.Month
scaler = MinMaxScaler()

scaler.fit(passengers.to_frame())
scaled_passengers = scaler.transform(passengers.to_frame())

In [40]:
def tokenize_month_info(input='1949-01'):
    year, month = input.split('-')
    diff = (int(year) - 1949)*12 + int(month)
    return torch.tensor(diff).to(torch.float32)

In [41]:
class AirlinePassengersDataset(Dataset):
    def __init__(self, transform=None):
        super().__init__()
        URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
        self.data = pd.read_csv(URL, index_col=False)
        self.scaler = MinMaxScaler()
        passengers_frame = self.data.Passengers.to_frame()
        self.scaler.fit(passengers_frame)
        scaled_passengers = self.scaler.transform(passengers_frame)
        self.passengers = scaled_passengers
        self.transform = transform

    
    def __getitem__(self, index):
        month, passenger = tokenize_month_info(self.data.Month[index]), torch.tensor(self.passengers[index]).to(torch.float32)
        return (month, passenger)
    
    def __len__(self):
        return len(self.data)


In [62]:
dataset = AirlinePassengersDataset()
split = int(len(dataset) * 0.9)
train_data, test_data = Subset(dataset, list(range(0, split))), Subset(dataset, list(range(split, len(dataset))))
loader = DataLoader(train_data, batch_size=4, shuffle=False)
test_loader = DataLoader(test_data, batch_size=len(dataset)-split, shuffle=False)

In [64]:
print(len(loader))
for word in test_loader:
    print(word[1])
    # print(word)
    break

33
tensor([[0.5849],
        [0.4981],
        [0.5811],
        [0.6042],
        [0.5541],
        [0.6081],
        [0.6892],
        [0.7104],
        [0.8320],
        [1.0000],
        [0.9691],
        [0.7799],
        [0.6892],
        [0.5521],
        [0.6332]])


<img src="/Users/musazenbilci/Desktop/mosesopposite/DeepLearningAI-RNN/W1A1/images/rnn_step_forward_figure2_v3a.png" style="width:700px;height:300px;">

In [44]:
class RNNCell(nn.Module):
    def __init__(self, input_size=1):
        super().__init__()
        self.linear1 = nn.Linear(input_size, input_size, bias=False) # Waa
        self.linear2 = nn.Linear(input_size, input_size, bias=True) # Wax + ba
        self.tanh = nn.Tanh()

        self.final = nn.Linear(input_size, input_size, bias=True) # Wya + by

    def forward(self, x, previous_a): # also called hidden state
        aa = self.linear1(previous_a)
        ax = self.linear2(x)
        x = torch.add(aa, ax)
        a = self.tanh(x)
        x = self.final(a)
        return x, a


In [49]:
initial_lr = 1e-1
cell = RNNCell()
optimizer = optim.SGD(cell.parameters(), lr=initial_lr)
criterion = torch.nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=8, threshold=0.0005)
a0 = torch.tensor([0]).to(torch.float32)

EPOCH=10000

for ep in range(EPOCH):
    epoch_loss = 0
    for batch in loader:
        batch_loss = 0
        optimizer.zero_grad()
        a_prev = a0
        passengers = batch[1]
        
        if len(passengers)<2:
            continue

        for p in range(len(passengers) - 1):
            logits, a = cell(passengers[p], a_prev)
            loss = criterion(logits, torch.tensor([passengers[p+1]]))
            batch_loss += loss
            a_prev = a

        batch_loss.backward()
        optimizer.step()
        # print("Batch avg loss", batch_loss.item() / 4)
        epoch_loss += batch_loss / 4

    print("EPOCH_LOSS", epoch_loss / len(loader))
    scheduler.step(epoch_loss / len(loader))
    if initial_lr != optimizer.param_groups[0]['lr']:
        print(f"LR Changed {optimizer.param_groups[0]['lr']}")
        initial_lr = optimizer.param_groups[0]['lr']
    

EPOCH_LOSS tensor(0.0399, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0264, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0236, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0219, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0199, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0178, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0157, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0140, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0124, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0111, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0099, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0089, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0081, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0074, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0067, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0062, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0058, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0055, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0052, grad_fn=<DivBackward0>)
EPOCH_LOSS tensor(0.0050, grad_fn=<DivBackward0>)


In [154]:
torch.save(cell.state_dict, './airlines_rnn_cell_loss21k.pth')

In [73]:
test_loader.dataset[0]

(tensor(130.), tensor([0.5849]))

In [83]:
torch.set_grad_enabled(False)
output = []
labels = []
input, a_prev = torch.tensor([test_loader.dataset[0][1]]), torch.tensor([0.]).to(torch.float32)

for i in range(1,14):
    logits, act = cell(input, torch.tensor([a_prev]))
    output.append(dataset.scaler.inverse_transform(logits.reshape(-1, 1)))
    labels.append(dataset.scaler.inverse_transform(test_loader.dataset[i][1].reshape(-1, 1)))
    input = logits
    a_prev = act
print(output)
print(labels)
torch.set_grad_enabled(True)


[array([[426.2286551]]), array([[442.42097163]]), array([[459.42326832]]), array([[476.82561505]]), array([[494.08118105]]), array([[510.56644154]]), array([[525.68195438]]), array([[538.96276987]]), array([[550.15555692]]), array([[559.23397899]]), array([[566.35574663]]), array([[571.78993726]]), array([[575.84592044]])]
[array([[361.99999839]]), array([[405.00000584]]), array([[416.99999428]]), array([[391.00001419]]), array([[418.9999975]]), array([[461.00000334]]), array([[472.0000056]]), array([[534.99999893]]), array([[622.]]), array([[606.00000513]]), array([[508.00000179]]), array([[461.00000334]]), array([[389.99999714]])]


In [ ]:
toutput = torch.tensor(output)
tlabels = torch.tensor(labels)
# mape (mean absolute percentage error)
# mean((actual-forecast) / actual)
mape = torch.mean(torch.abs((torch.squeeze(toutput)-torch.squeeze(tlabels)) / torch.squeeze(tlabels))) # 0.1556
mape

# mae (mean absolute error)
mae = torch.mean(torch.abs((torch.squeeze(toutput)-torch.squeeze(tlabels)))) # 68.1379
mae

# rmse (root mean square error)
rmse = torch.sqrt(torch.mean(torch.square((torch.squeeze(toutput)-torch.squeeze(tlabels))))) # 80.0241
rmse


tensor(80.0241, dtype=torch.float64)